# Example of DOV search methods for interpretations (informele hydrogeologische stratigrafie)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/DOV-Vlaanderen/pydov/master?filepath=docs%2Fnotebooks%2Fsearch_informele_hydrostratigrafie.ipynb)

## Use cases explained below
* Get 'informele hydrogeologische stratigrafie' in a bounding box
* Get 'informele hydrogeologische stratigrafie' with specific properties
* Get 'informele hydrogeologische stratigrafie' in a bounding box based on specific properties
* Select 'informele hydrogeologische stratigrafie' in a municipality and return date
* Get 'informele hydrogeologische stratigrafie' based on fields not available in the standard output dataframe
* Get 'informele hydrogeologische stratigrafie' data, returning fields not available in the standard output dataframe

In [1]:
%matplotlib inline
import inspect, sys

In [2]:
# check pydov path
import pydov

## Get information about the datatype 'Informele hydrogeologische stratigrafie'

In [3]:
from pydov.search.interpretaties import InformeleHydrogeologischeStratigrafieSearch
itp = InformeleHydrogeologischeStratigrafieSearch()

A description is provided for the 'Informele stratigrafie' datatype:

In [4]:
itp.get_description()

'Een informele hydrogeologische stratigrafie van een boring geeft, op basis van de (gecodeerde) lithologie, een indeling weer naar de al dan niet watervoerende eigenschappen van een bepaald beschreven diepte-interval. \r\nHet verschil met de hydrogeologische stratigrafie is dat voor de informele hydrogeologische stratigrafie geen algemene standaard gebruikt wordt.  Bij de hydrogeologische stratigrafie wordt de Hydrogeologische Codering van de Ondergrond van Vlaanderen (HCOV). Deze kan beschouwd worden als de officiele hydrogeologische codering voor het Vlaams Gewest.'

The different fields that are available for objects of the 'Informele hydrogeologische stratigrafie' datatype can be requested with the get_fields() method:

In [5]:
fields = itp.get_fields()

# print available fields
for f in fields.values():
    print(f['name'])

pkey_interpretatie
Type_proef
Proefnummer
pkey_boring
x
y
start_interpretatie_mtaw
diepte_tot_m
gemeente
Auteurs
Datum
Opdrachten
betrouwbaarheid_interpretatie
Geldig_van
Geldig_tot
eerste_invoer
geom
diepte_laag_van
diepte_laag_tot
beschrijving


You can get more information of a field by requesting it from the fields dictionary:

* *name*: name of the field
* *definition*: definition of this field
* *cost*: currently this is either 1 or 10, depending on the datasource of the field. It is an indication of the expected time it will take to retrieve this field in the output dataframe.
* *notnull*: whether the field is mandatory or not
* *type*: datatype of the values of this field
* *codelist*: optionally, a codelist that describes the possible values of this field

Alternatively, you can list all the fields and their details by inspecting the `get_fields()` output or the search instance itself in a notebook:

In [6]:
itp

## Example use cases

### Get 'informele hydrogeologische stratigrafie' in a bounding box

Get data for all the 'informele hydrogeologisch stratigrafie' interpretations that are geographically located within the bounds of the specified box.

The coordinates are in the Belgian Lambert72 (EPSG:31370) coordinate system and are given in the order of lower left x, lower left y, upper right x, upper right y.

In [7]:
from pydov.util.location import Within, Box

df = itp.search(location=Within(Box(100000, 100000, 200000, 200000)))
df.head()

[000/001] .
[000/001] .


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,beschrijving
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,0.0,2.5,QUARTAIR
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,2.5,13.6,FORMATIE VAN MALDEGEM
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,13.6,25.5,FORMATIE VAN MALDEGEM (Lid van Wemmel)
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,25.5,33.2,FORMATIE VAN LEDE
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,33.2,67.3,FORMATIE VAN BRUSSEL


The dataframe contains one 'informele stratigrafie' interpretation where three layers ('laag') were identified. The available data are flattened to represent unique attributes per row of the dataframe.

Using the *pkey_interpretatie* field one can request the details of this interpretation in a webbrowser:

In [8]:
for pkey_interpretatie in set(df.pkey_interpretatie):
    print(pkey_interpretatie)

https://www.dov.vlaanderen.be/data/interpretatie/1981-217464


### Get 'informele hydrogeologische stratigrafie' with specific properties

Next to querying interpretations based on their geographic location within a bounding box, we can also search for interpretations matching a specific set of properties. For this we can build a query using a combination of the 'InformeleHydrogeologischeStratigrafie' fields and operators provided by the WFS protocol.

A list of possible operators can be found below:

In [9]:
[i for i,j in inspect.getmembers(sys.modules['owslib.fes2'], inspect.isclass) if 'Property' in i]

['PropertyIsBetween',
 'PropertyIsEqualTo',
 'PropertyIsGreaterThan',
 'PropertyIsGreaterThanOrEqualTo',
 'PropertyIsLessThan',
 'PropertyIsLessThanOrEqualTo',
 'PropertyIsLike',
 'PropertyIsNotEqualTo',
 'PropertyIsNull',
 'SortProperty']

In this example we build a query using the *PropertyIsEqualTo* operator to find all interpretations that are within the community (gemeente) of 'Herstappe':

In [10]:
from owslib.fes2 import PropertyIsEqualTo

query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Rotselaar')
df = itp.search(query=query)

df.head()

[000/001] .
[000/001] c


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,beschrijving
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,0.0,2.5,QUARTAIR
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,2.5,13.6,FORMATIE VAN MALDEGEM
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,13.6,25.5,FORMATIE VAN MALDEGEM (Lid van Wemmel)
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,25.5,33.2,FORMATIE VAN LEDE
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,33.2,67.3,FORMATIE VAN BRUSSEL


Once again we can use the *pkey_interpretatie* as a permanent link to the information of these interpretations:

In [11]:
for pkey_interpretatie in set(df.pkey_interpretatie):
    print(pkey_interpretatie)

https://www.dov.vlaanderen.be/data/interpretatie/1981-217464


### Get 'informele hydrogeologische stratigrafie' in a bounding box based on specific properties

We can combine a query on attributes with a query on geographic location to get the interpretations within a bounding box that have specific properties.

The following example requests the interpretations of boreholes only, within the given bounding box.

(Note that the datatype of the *literal* parameter should be a string, regardless of the datatype of this field in the output dataframe.)

In [12]:
from owslib.fes2 import PropertyIsEqualTo

query = PropertyIsEqualTo(
            propertyname='Type_proef',
            literal='Boring')

df = itp.search(
    location=Within(Box(100000, 100000, 200000, 200000)),
    query=query
    )

df.head()

[000/001] .
[000/001] c


,pkey_interpretatie,pkey_boring,betrouwbaarheid_interpretatie,x,y,start_interpretatie_mtaw,diepte_laag_van,diepte_laag_tot,beschrijving
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,0.0,2.5,QUARTAIR
1,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,2.5,13.6,FORMATIE VAN MALDEGEM
2,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,13.6,25.5,FORMATIE VAN MALDEGEM (Lid van Wemmel)
3,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,25.5,33.2,FORMATIE VAN LEDE
4,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,goed,172148.5,182587.5,10.94,33.2,67.3,FORMATIE VAN BRUSSEL


We can look at one of the interpretations in a webbrowser using its *pkey_interpretatie*:

In [13]:
for pkey_interpretatie in set(df.pkey_interpretatie):
    print(pkey_interpretatie)

https://www.dov.vlaanderen.be/data/interpretatie/1981-217464


### Select 'informele hydrogeologische stratigrafie' in a municipality and return date

We can limit the columns in the output dataframe by specifying the *return_fields* parameter in our search.

In this example we query all the 'informele stratigrafie' interpretations in the community of Rotselaar and return their date:

In [14]:
query = PropertyIsEqualTo(propertyname='gemeente',
                          literal='Rotselaar')
df = itp.search(query=query,
                return_fields=('Datum',))
df.head()

[000/001] .


,Datum
0,1981-01-01


In [15]:
df.describe()

,Datum
count,1
unique,1
top,1981-01-01
freq,1


### Get 'informele hydrogeologische stratigrafie' based on fields not available in the standard output dataframe

To keep the output dataframe size acceptable, not all available WFS fields are included in the standard output. However, one can use this information to select interpretations as illustrated below.

For example, make a selection of the interpretations in the municipality of Rotselaar, before 1/1/1990:

In [16]:
from owslib.fes2 import And, PropertyIsEqualTo, PropertyIsLessThan

query = And([PropertyIsEqualTo(propertyname='gemeente',
                               literal='Rotselaar'),
             PropertyIsLessThan(propertyname='Datum', 
                                 literal='1990-01-01')]
            )
df = itp.search(query=query,
                return_fields=('pkey_interpretatie', 'Datum'))
df.head()

[000/001] .


,pkey_interpretatie,Datum
0,https://www.dov.vlaanderen.be/data/interpretat...,1981-01-01


### Get 'informele hydrogeologische stratigrafie' data, returning fields not available in the standard output dataframe

As denoted in the previous example, not all available fields are available in the default output frame to keep its size limited. However, you can request any available field by including it in the *return_fields* parameter of the search:

In [17]:
query = PropertyIsEqualTo(
            propertyname='gemeente',
            literal='Rotselaar')

df = itp.search(query=query,
                return_fields=('pkey_interpretatie', 'pkey_boring',
                               'x', 'y', 'start_interpretatie_mtaw', 'gemeente', 'Auteurs', 'Proefnummer'))

df.head()

[000/001] .


,pkey_interpretatie,pkey_boring,x,y,start_interpretatie_mtaw,gemeente,Auteurs,Proefnummer
0,https://www.dov.vlaanderen.be/data/interpretat...,https://www.dov.vlaanderen.be/data/boring/1981...,172148.5,182587.5,10.94,Rotselaar,Onbekend - Vlaamse Maatschappij voor Watervoor...,B/3005-016


## Visualize results

Using Folium, we can display the results of our search on a map.

In [18]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Transformer

In [19]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    transformer = Transformer.from_crs("epsg:31370", "epsg:4326", always_xy=True)
    x2,y2 = transformer.transform(x1, y1)
    return x2, y2

df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y'])) 
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [20]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=12)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['Proefnummer'][loc]).add_to(marker_cluster)
fmap
